## Imports

In [37]:
import time
from typing import Optional, List, Tuple
import imageio
import cv2
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

## Detect and convert image colors

In [38]:
def threshold_rel(img, lo, hi):
    vmin = np.min(img)
    vmax = np.max(img)
    
    vlo = vmin + (vmax - vmin) * lo
    vhi = vmin + (vmax - vmin) * hi
    return np.uint8((img >= vlo) & (img <= vhi)) * 255

def threshold_abs(img, lo, hi):
    return np.uint8((img >= lo) & (img <= hi)) * 255


def getHLSImage(img): 
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    v_channel = hsv[:,:,2]

    right_lane = threshold_rel(l_channel, 0.8, 1.0)
    right_lane[:,:750] = 0

    left_lane = threshold_abs(h_channel, 20, 30)
    left_lane &= threshold_rel(v_channel, 0.7, 1.0)
    left_lane[:,550:] = 0

    finalImg = left_lane | right_lane
    return finalImg


def plotImage(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))


## Canny Edge Detection

In [39]:
# h1 = 100
# h2 = 200
def cannyEdgeDetection(img, h1 = 160, h2= 170):
    blurredImg = cv2.GaussianBlur(img, (5,5), 0)
    cannyImg = cv2.Canny(blurredImg, h1 ,h2)
    return cannyImg

## Image Masking

In [40]:
def regionOfInterest(img, color):
    height = img.shape[0]
    width = img.shape[1]
    regionOfInterest = [
        (0, height),
        (width/2, height/2),
        (width, height)
    ]
    rectangle = np.array([regionOfInterest], np.int32)
    mask = np.zeros_like(img)
    # all pixels will be black
    cv2.fillPoly(mask, rectangle, color)
    return mask

In [41]:
def maskImage(img):
    mask = regionOfInterest(img, 255)
    # plt.imshow(cv2.cvtColor(mask, cv2.COLOR_BGR2RGB))
    maskedImage = cv2.bitwise_and(img, mask)
    return maskedImage

## Perspective Transform
Applying transform to the image to see it in the bird's eye view

In [42]:
""" Assigning the source and destination points for the perspective transform """
# coordinates = [x, y]
top_left = [510, 480]
top_right = [780, 480]
bottom_left = [270, 690]
bottom_right = [1230, 690]


srcPoints = np.array([
    top_left,
    top_right,
    bottom_left,
    bottom_right,

]).astype(np.float32)

dstPoints = np.array([
    [0, 0],
    [550, 0],
    [0, 350],
    [550, 350],
]).astype(np.float32)

warpedImgSize = (550, 350) # (width, height) for transformed image

''' Perspective transform functions for the lane detection '''
def perspectiveTransform(srcPoints, dstPoints):
    M = cv2.getPerspectiveTransform(srcPoints, dstPoints)
    Minv = cv2.getPerspectiveTransform(dstPoints, srcPoints)
    return M, Minv


def warpPerspective(img, imgSize, M):
    return cv2.warpPerspective(img, M, imgSize, cv2.INTER_LINEAR)


def transformImage(img, source, destination, imgSize):
    """ transform the image to bird's eye view """

    M, Minv = perspectiveTransform(source, destination)
    warpedImg = warpPerspective(img, imgSize, M)
    # plotImage(warpedImg)
    return warpedImg


## Hough Transform

In [43]:
def getPoints(I,numPts):
    """ manually select points"""
    %matplotlib
    fig,ax = plt.subplots(1,figsize=(15,30))
    plt.imshow(I,cmap='gray')
    pts = np.round(np.array(plt.ginput(n=numPts)))
    pts = pts[:,[1,0]].T
    plt.close()
    return pts

In [44]:

def hough_lines(
    img:np.ndarray,
    rho: Optional[int] = 2,
    theta: Optional[float] = np.pi/180,
    threshold : Optional[int] = 50,
    min_line_len: Optional[int] = 5,
    max_line_gap: Optional[int] = 5)\
        -> List[np.ndarray]:
    """
    takes in an image and returns its hough lines

    Args:
    -------
        img (numpy.ndarray): an image
        rho (int): the resolution of the rho axis
        theta (float): the resolution of the theta axis
        threshold (int): the minimum number of votes a line needs to be considered
        min_line_len (int): the minimum length of a line
        max_line_gap (int): the maximum gap between two lines

    Returns:
    ---------
    A list of hough lines
    
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

def draw_lines(
        img: np.ndarray,
        lines: np.ndarray,
        color:Optional[Tuple[int,int,int]]=[0, 255, 0]
        , thickness: Optional[int]=3)\
            -> np.ndarray:
    """
    takes in an image and returns an image with the lines drawn on an empty image

    Args:
    -------
        img (numpy.ndarray): an image
        lines (numpy.ndarray): a list of hough lines
        color (Tuple[int,int,int]): the color of the lines
        thickness (int): the thickness of the lines

    Returns:
    ---------
    An image with the lines drawn on it
    
    """
    
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    if lines is not None:
        [
            [
                cv2.line(line_img, (x1, y1), (x2, y2), color, thickness)
                for x1, y1, x2, y2 in line
            ]
            for line in lines
        ]
    return line_img

def hough_image(
    img: np.ndarray
    ,img_hough: np.ndarray)\
        -> np.ndarray:
    """
    takes in an image and its lines and returns it with the hough lines drawn on it

    Args:
    -------
        img (numpy.ndarray): an image
        img_hough (numpy.ndarray): an image with the hough lines drawn on it

    Returns:
    ---------
    An image with the hough lines drawn on it
    
    """
    return cv2.addWeighted(img, 0.8, img_hough, 1, 0)


## Pipeline

In [88]:
# Video Handlers: one to read input video and other for generating output video
vid = imageio.get_reader('../../videos/project_video.mp4', 'ffmpeg')
finalVid = imageio.get_writer('./output_project_video3.mp4', fps = 30)

for i, img in enumerate(vid):
    # Applying filters to the image
    HLSImg = getHLSImage(img)
    # print(HLSImg.shape)
    cannyImg = cannyEdgeDetection(HLSImg)
    maskedImage = maskImage(cannyImg)
    transformedImage = transformImage(maskedImage, srcPoints, dstPoints, warpedImgSize)
    # plt.imshow(transformedImage)
    lines = hough_lines(img=transformedImage)
    # print(lines)
    # plt.imshow(lines)
    lineImg = draw_lines(HLSImg, lines)
    warpedImgSize = (1280, 720)
    lineTransformedImage = transformImage(lineImg, dstPoints, srcPoints, warpedImgSize)
    # print(lineTransformedImage.shape)
    houghImage = hough_image(img, lineTransformedImage)
    # result = cv2.addWeighted(img, 1, mask, 0.2, 0)

    
    # Appending final image to the video
    finalVid.append_data(houghImage)


finalVid.close()